# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 1)

### 1. Problem Description:
"In a city of your choice, if someone is looking open their own business, where would you recommend that they setup their office?"


### 2. Data Description:
Toronto is the capital city Canada. Choosing a neighborhood to live in or open a business is difficult task that requires informed decision based on location data. The approach to this problem will include the following: 
- Scrape the Wikipedia page
- Wrangle the data, clean it
- Read it into a pandas dataframe
- Explore and cluster the neighborhoods

In [12]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

#to convert address into latitude and longitude values
from geopy.geocoders import Nominatim 

#transform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

#map rendering library
import folium

#import k-means from clustering stage
from sklearn.cluster import KMeans

#matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('libraries imported')

libraries imported


In [13]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'html5lib')

#using provided sample code
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [14]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [15]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
print("Shape: ", df.shape)

Shape:  (103, 3)
